In [41]:
# 사전 리스트
import pandas as pd
df_hawkish = pd.read_csv('df_hawkish_dict.csv')
df_dovish = pd.read_csv('df_dovish_dict.csv')
dic_hawkish = list(df_hawkish['ngram'])
dic_dovish = list(df_dovish['ngram'])

In [6]:
from tqdm.notebook import tqdm

In [19]:
import pandas as pd 
import numpy as np
import re

In [42]:
df_dic = pd.read_csv('ngramized_minutes_test.csv')#n_gram 만든 라벨링된 의사록
#-------------------------------필요없는 컬럼 제거하기=---------------------------------
df_dic = df[['ID', 'date', 'sent', 'n_gram']]

In [80]:
#------------------------------n_gram split했을 때 이쁘게 뽑히게하기-----------------
for i in range(len(df_dic)):
    df_dic['n_gram'].iloc[i] = df_dic['n_gram'].iloc[i].replace("[","").replace("]","").replace("'","").replace(",","").split()

C:\Users\180309\anaconda3\envs\mulcam_b\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [94]:
df_dic['hawkish'] = np.zeros(len(df_dic))
df_dic['dovish'] = np.zeros(len(df_dic))

In [95]:
from tqdm.notebook import tqdm
for i in tqdm(range(len(df_dic))):
    for word in df_dic['n_gram'].iloc[i]:
        if word in dic_hawkish:
            df_dic['hawkish'].iloc[i] += 1

In [96]:
from tqdm.notebook import tqdm
for i in tqdm(range(len(df_dic))):
    for word in df_dic['n_gram'].iloc[i]:
        if word in dic_dovish:
            df_dic['dovish'].iloc[i] += 1

C:\Users\180309\anaconda3\envs\mulcam_b\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [97]:
df_dic[:5]

,ID,date,sent,n_gram,hawkish,dovish
0,2005년도 제10차,2005-05-12,['일부 위원은 내외금리차 역전에 따라 자금의 해외유출이 대규모로 발생될 가능성을 ...,"[내외/NNG, 금리갭/NNG, 역전/NNG, 자금/NNG, 해외/NNG, 유출/N...",4.0,0.0
1,2005년도 제10차,2005-05-12,"', '이에 대해 관련부서에서는 우리나라에서는 그러한 사례가 없었기 때문에 대만과 ...","[대해/VV, 그러/NNG, 사례/NNG, 없/VA, 사례/NNG, 살펴보/VV, ...",3.0,1.0
2,2005년도 제10차,2005-05-12,"', '동 위원은 자금의 해외유출에 대한 정책적 대응을 위해 내외금리나 환율에 대해...","[자금/NNG, 해외/NNG, 유출/NNG, 정책/NNG, 대응/NNG, 위하/VV...",2.0,0.0
3,2005년도 제10차,2005-05-12,"', '다른 일부 위원은 최근 금융기관의 외화대출이 늘어나고 있는 것과 관련하 여 ...","[금융기관/NNG, 외화대출/NNG, 늘/VV, 금융기관/NNG, 해외/NNG, 차...",0.0,2.0
4,2005년도 제10차,2005-05-12,"', '또 다른 일부 위원은 전년 동기와 비교할 때 금년 1～4월중 국내시장에 대 ...","[또/MAG, 비교/NNG, 하/VV, 때/NNG, 중/NNG, 시장/NNG, 대/...",9.0,1.0


In [99]:
polarity = df_dic.groupby(df_dic['date']).sum()
polarity

,hawkish,dovish
date,,
2005-05-12,449.0,53.0
2005-06-09,461.0,59.0
2005-07-07,498.0,84.0
2005-08-11,610.0,66.0
2005-09-08,551.0,73.0
...,...,...
2017-05-25,100.0,26.0
2017-07-13,125.0,13.0
2017-08-31,141.0,19.0


In [103]:
#-----------------------------------0인값확인하기-------------------------------------
polarity[polarity['dovish'] == 0],polarity[polarity['hawkish'] == 0]

,hawkish,dovish
date,,
2005-05-12,449.0,53.0
2014-04-10,225.0,53.0
2016-03-10,212.0,53.0


In [104]:
polarity['polarity_score'] = (polarity['hawkish']-polarity['dovish']) / (polarity['hawkish'] + polarity['dovish'])

In [105]:
#---------------------------------기준금리 합치기---------------------------------------

In [106]:
base_rate = pd.read_csv('df_baserate.csv')
base_rate[:5]

,날짜,기준금리
0,2005-05-01,3.25
1,2005-05-02,3.25
2,2005-05-03,3.25
3,2005-05-04,3.25
4,2005-05-05,3.25


In [107]:
df_new = polarity.merge(base_rate, left_on='date', right_on='날짜', how='left')
df_new[:5]

,hawkish,dovish,polarity_score,날짜,기준금리
0,449.0,53.0,0.788845,2005-05-12,3.25
1,461.0,59.0,0.773077,2005-06-09,3.25
2,498.0,84.0,0.711340,2005-07-07,3.25
3,610.0,66.0,0.804734,2005-08-11,3.25
4,551.0,73.0,0.766026,2005-09-08,3.25


In [108]:
df_new.corr()

,hawkish,dovish,polarity_score,기준금리
hawkish,1.000000,0.660538,0.298684,0.361079
dovish,0.660538,1.000000,-0.426241,0.250770
polarity_score,0.298684,-0.426241,1.000000,0.112312
기준금리,0.361079,0.250770,0.112312,1.000000
